# **Aggregation of Dataframes and Manipulation of Timestamps**

## _ETL Workflow Notebook 1_

## Content:
1. Applying a list of row filters;
2. Merging on timestamp;
3. Merging (joining) dataframes on given keys; and sorting the merged table;
4. Record linkage: fuzzy merging (joining) of dataframes on similar strings;
5. Concatenating (SQL Union/Stacking/Appending) dataframes;
6. Dataframe general characterization;
7. Dropping specific columns or rows from the dataframe;
8. Removing duplicate rows from the dataframe;
9. Removing all columns and rows that contain only missing values;
10. Grouping by timestamp;
11. Grouping by a given variable;
12. Extracting timestamp information;
13. Calculating differences between successive timestamps (delays);
14. Calculating timedeltas;
15. Adding or subtracting timedeltas;
16. Slicing the dataframe (selecting a specific subset of rows).

Marco Cesar Prado Soares, Data Scientist Specialist - Bayer Crop Science LATAM
- marcosoares.feq@gmail.com
- marco.soares@bayer.com

## **Load Python Libraries in Global Context**

In [2]:
import load
from idsw import *

/content/idsw/datafetch/core.py:1713: SyntaxWarning: invalid escape sequence '\p'
  engine = create_engine(r"sqlite:///C:\path\to\foo.db")
/content/idsw/datafetch/pipes.py:317: SyntaxWarning: invalid escape sequence '\s'
  txt_csv_col_sep = '\s+'; or txt_csv_col_sep = '\t' (in this last example, the tabulation
/content/idsw/datafetch/pipes.py:1271: SyntaxWarning: invalid escape sequence '\('
  To find a spec like "TensorSpec(shape=(None, *, *)" the Regex would be: r"TensorSpec\(shape=\(None, (\d+), (\d+)\)"
/content/idsw/modelling/core.py:2751: SyntaxWarning: invalid escape sequence '\{'
  * Given a training set $\{x^{(1)}, ..., x^{(m)}\}$ you want to estimate the Gaussian distribution for each
/content/idsw/modelling/core.py:2846: SyntaxWarning: invalid escape sequence '\{'
  $\{(x_{\rm cv}^{(1)}, y_{\rm cv}^{(1)}),\ldots, (x_{\rm cv}^{(m_{\rm cv})}, y_{\rm cv}^{(m_{\rm cv})})\}$,


Package copied to the working directory.
To import its whole content, run:

    from idsw import *



/content/idsw/modelling/deep.py:239: SyntaxWarning: invalid escape sequence '\%'
  print("In this example, we added Dropout(0.5). It means that you lose 50\% of nodes. If using Dropout(0.2), you would lose 20\% of nodes.")
/content/idsw/modelling/deep.py:445: SyntaxWarning: invalid escape sequence '\%'
  print("In this example, we added Dropout(0.5). It means that you lose 50\% of nodes. If using Dropout(0.2), you would lose 20\% of nodes.")
/content/idsw/etl/core.py:2462: SyntaxWarning: invalid escape sequence '\d'
  r'st\d\s\w{3,10}'
/content/idsw/etl/core.py:2583: SyntaxWarning: invalid escape sequence '\d'
  - re.findall(r"\d+-\d+", text) - returns: ['4-3', '10-04']
/content/idsw/etl/core.py:2632: SyntaxWarning: invalid escape sequence '\s'
  - If we do re.findall(r"the\s\d+s", my_string) - returns: ['the 80s', 'the 90s']
/content/idsw/etl/core.py:2720: SyntaxWarning: invalid escape sequence '\d'
  - Example: re.match(r"\d+", "12345bcada") - returns: <_sre.SRE_Match object; span=(0

In [3]:
! pip install ucimlrepo
def download_dataset():

  from ucimlrepo import fetch_ucirepo
  # Forty Soybean Cultivars from Subsequent Harvests
  #https://archive.ics.uci.edu/dataset/913/forty+soybean+cultivars+from+subsequent+harvests

  # fetch dataset
  forty_soybean_cultivars_from_subsequent_harvests = fetch_ucirepo(id=913)

  # data (as pandas dataframes)
  X = forty_soybean_cultivars_from_subsequent_harvests.data.features
  y = forty_soybean_cultivars_from_subsequent_harvests.data.targets

  # metadata
  print(forty_soybean_cultivars_from_subsequent_harvests.metadata)

  # variable information
  print(forty_soybean_cultivars_from_subsequent_harvests.variables)

  return X

dataset = download_dataset()

{'uci_id': 913, 'name': 'Forty Soybean Cultivars from Subsequent Harvests', 'repository_url': 'https://archive.ics.uci.edu/dataset/913/forty+soybean+cultivars+from+subsequent+harvests', 'data_url': 'https://archive.ics.uci.edu/static/public/913/data.csv', 'abstract': 'Soybean cultivation is one of the most important because it is used in several segments of the food industry. The evaluation of soybean cultivars subject to different planting and harvesting characteristics is an ongoing field of research. We present a dataset obtained from forty soybean cultivars planted in subsequent seasons. The experiment used randomized blocks, arranged in a split-plot scheme, with four replications. The following variables were collected: plant height, insertion of the first pod, number of stems, number of legumes per plant, number of grains per pod, thousand seed weight, and grain yield, resulting in 320 data samples. The dataset presented can be used by researchers from different fields of activit

## **Call the functions**

### **Characterizing the dataframe**

In [4]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

#New dataframes saved as df_shape, df_columns_list, df_dtypes, df_general_statistics, df_missing_values.
# Simply modify this object on the left of equality:
df_shape, df_columns_array, df_dtypes, df_general_statistics, df_missing_values = df_general_characterization (df = DATASET)

Dataframe's 10 first rows:



,Season,Cultivar,Repetition,PH,IFP,NLP,NGP,NGL,NS,MHG,GY
0,1,NEO 760 CE,1,58.80,15.20,98.2,177.80,1.81,5.20,152.20,3232.82
1,1,NEO 760 CE,2,58.60,13.40,102.0,195.00,1.85,7.20,141.69,3517.36
2,1,NEO 760 CE,3,63.40,17.20,100.4,203.00,2.02,6.80,148.81,3391.46
3,1,NEO 760 CE,4,60.27,15.27,100.2,191.93,1.89,6.40,148.50,3312.58
4,1,MANU IPRO,1,81.20,18.00,98.8,173.00,1.75,7.40,145.59,3230.99
5,1,MANU IPRO,2,75.80,20.80,69.2,128.00,1.85,7.20,154.87,3374.80
6,1,MANU IPRO,3,84.40,15.80,95.4,161.80,1.70,6.80,150.23,3182.76
7,1,MANU IPRO,4,80.47,18.20,87.8,154.27,1.77,7.13,149.90,3165.72
8,1,77HO111I2X - GUAPORÉ,1,52.20,14.40,64.8,148.80,2.30,7.20,180.25,3640.46
9,1,77HO111I2X - GUAPORÉ,2,55.80,15.80,72.0,188.80,2.62,6.20,176.75,3602.34




Dataframe's 10 last rows:



,Season,Cultivar,Repetition,PH,IFP,NLP,NGP,NGL,NS,MHG,GY
310,2,NK 8770 IPRO,3,91.40,18.20,60.20,141.40,2.35,3.60,166.68,3150.862069
311,2,NK 8770 IPRO,4,89.60,16.27,70.93,151.13,2.12,3.00,160.81,3037.189655
312,2,FTR 4288 IPRO,1,82.60,16.00,84.00,147.20,1.75,3.60,146.98,3475.448276
313,2,FTR 4288 IPRO,2,87.60,15.80,79.20,151.80,1.92,3.80,139.22,3409.103448
314,2,FTR 4288 IPRO,3,94.80,17.20,64.00,118.00,1.84,3.60,135.65,3304.678161
315,2,FTR 4288 IPRO,4,88.33,16.33,75.73,139.00,1.84,3.67,135.19,3348.931034
316,2,FTR 3190 IPRO,1,64.40,16.60,76.00,168.00,2.21,3.60,145.69,3418.850575
317,2,FTR 3190 IPRO,2,64.60,17.60,116.80,271.20,2.32,3.80,147.24,3651.586207
318,2,FTR 3190 IPRO,3,58.80,14.80,86.40,180.60,2.09,2.20,156.32,3487.931034
319,2,FTR 3190 IPRO,4,62.60,16.33,93.07,206.60,2.21,3.20,157.61,3605.310345




Dataframe's shape = (number of rows, number of columns) =



...

,dtype_series
dataframe_column,
Season,int64
Cultivar,object
Repetition,int64
PH,float64
IFP,float64
NLP,float64
NGP,float64
NGL,float64
NS,float64




Dataframe's general (summary) statistics for numeric variables:



dataframe_column,Season,Repetition,PH,IFP,NLP,NGP,NGL,NS,MHG,GY
count,320.000000,320.000000,320.000000,320.0000,320.000000,320.000000,320.000000,320.000000,320.000000,320.000000
mean,1.500000,2.500000,68.386781,15.4650,59.088313,135.085844,2.290844,4.071656,168.322313,3418.553794
std,0.500783,1.119785,8.958194,3.0243,20.068187,60.494529,0.840116,1.474531,19.625566,503.003602
min,1.000000,1.000000,47.600000,7.2000,20.200000,47.800000,0.940000,0.400000,127.060000,1538.230000
25%,1.000000,1.750000,62.950000,13.6000,44.350000,95.052500,2.000000,3.000000,153.845000,3126.611552
50%,1.500000,2.500000,67.200000,15.6000,54.500000,123.000000,2.280000,3.800000,166.150000,3397.276724
75%,2.000000,3.250000,74.347500,17.3300,71.220000,161.350000,2.480000,5.000000,183.182500,3708.262931
max,2.000000,4.000000,94.800000,26.4000,123.000000,683.400000,14.860000,9.000000,216.000000,4930.000000




Missing values on each feature; and missingness considering all rows from the dataframe:
(note: 'missingness_accross_rows' was calculated by: checking which rows have at least one missing value (NA); and then comparing total rows with NAs with total rows in the dataframe).



,count_of_missing_values,proportion_of_missing_values,percent_of_missing_values
dataframe_column,,,
Season,0,0.0,0.0
Cultivar,0,0.0,0.0
Repetition,0,0.0,0.0
PH,0,0.0,0.0
IFP,0,0.0,0.0
NLP,0,0.0,0.0
NGP,0,0.0,0.0
NGL,0,0.0,0.0
NS,0,0.0,0.0


### **Removing duplicate rows from the dataframe**

In [6]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

LIST_OF_COLUMNS_TO_ANALYZE = None
# if LIST_OF_COLUMNS_TO_ANALYZE = None, the whole dataset will be analyzed, i.e., rows
# will be removed only if they have same values for all columns from the dataset.
# Alternatively, pass a list of columns names (strings), if you want to remove rows with
# same values for that combination of columns. Pass it as a list, even if there is a single column
# being declared.
# e.g. LIST_OF_COLUMNS_TO_ANALYZE = ['column1'] will check only 'column1'. Entries with same value
# on 'column1' will be considered duplicates and will be removed.
# LIST_OF_COLUMNS_TO_ANALYZE = ['col1', 'col2',  'col3'] will analyze the combination of 3 columns:
# 'col1', 'col2', and 'col3'. Only rows with same value for these 3 columns will be considered
# duplicates and will be removed.

WHICH_ROW_TO_KEEP = 'first'
# WHICH_ROW_TO_KEEP = 'first' will keep the first detected row and remove all other duplicates. If
# None or an invalid string is input, this method will be selected.
# WHICH_ROW_TO_KEEP = 'last' will keep only the last detected duplicate row, and remove all the others.

RESET_INDEX_AFTER_DROP = True
# RESET_INDEX_AFTER_DROP = True. keep it True to restarting the indexing numeration after dropping.
# Alternatively, set RESET_INDEX_AFTER_DROP = False to keep the original numeration (the removed indices
# will be missing).

# New dataframe saved as cleaned_df. Simply modify this object on the left of equality:
dataset = remove_duplicate_rows (df = DATASET, list_of_columns_to_analyze = LIST_OF_COLUMNS_TO_ANALYZE, which_row_to_keep = WHICH_ROW_TO_KEEP, reset_index_after_drop = RESET_INDEX_AFTER_DROP)

The rows with duplicate entries were successfully removed.
Only the first one of the duplicate entries was kept in the dataset.

The indices of the dataset were successfully restarted.


            Initial number of rows: 320
            Final number of rows: 320
            Eliminated 0 rows (0.00 % of the rows.)
            
Check the 10 first rows from the returned dataset:



,Season,Cultivar,Repetition,PH,IFP,NLP,NGP,NGL,NS,MHG,GY
0,1,NEO 760 CE,1,58.80,15.20,98.2,177.80,1.81,5.20,152.20,3232.82
1,1,NEO 760 CE,2,58.60,13.40,102.0,195.00,1.85,7.20,141.69,3517.36
2,1,NEO 760 CE,3,63.40,17.20,100.4,203.00,2.02,6.80,148.81,3391.46
3,1,NEO 760 CE,4,60.27,15.27,100.2,191.93,1.89,6.40,148.50,3312.58
4,1,MANU IPRO,1,81.20,18.00,98.8,173.00,1.75,7.40,145.59,3230.99
5,1,MANU IPRO,2,75.80,20.80,69.2,128.00,1.85,7.20,154.87,3374.80
6,1,MANU IPRO,3,84.40,15.80,95.4,161.80,1.70,6.80,150.23,3182.76
7,1,MANU IPRO,4,80.47,18.20,87.8,154.27,1.77,7.13,149.90,3165.72
8,1,77HO111I2X - GUAPORÉ,1,52.20,14.40,64.8,148.80,2.30,7.20,180.25,3640.46
9,1,77HO111I2X - GUAPORÉ,2,55.80,15.80,72.0,188.80,2.62,6.20,176.75,3602.34


### **Removing all columns and rows that contain only missing values**

In [7]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

LIST_OF_COLUMNS_TO_IGNORE = None
# list_of_columns_to_ignore: if you do not want to check a specific column, pass its name
# (header) as an element from this list. It should be declared as a list even if it contains
# a single value.
# e.g. list_of_columns_to_ignore = ['column1'] will not analyze missing values in column named
# 'column1'; list_of_columns_to_ignore = ['col1', 'col2'] will ignore columns 'col1' and 'col2'

# Cleaned dataframe returned as cleaned_df.
# Simply modify this object on the left of equality:
dataset = remove_completely_blank_rows_and_columns (df = DATASET, list_of_columns_to_ignore = LIST_OF_COLUMNS_TO_IGNORE)

0 rows (0.00 % of the rows) were completely blank and were removed.
0 columns were completely blank and were removed.

No blank columns or rows were found. Returning the original dataframe.



### **Characterizing the categorical variables**

In [8]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

# Dataframe with summary from the categorical variables returned as cat_vars_summary.
# Simply modify this object on the left of equality:
cat_vars_summary = characterize_categorical_variables (df = DATASET)



Finished analyzing the categorical variables. Check the summary dataframe:



,value,counts_of_occurences,percent_of_occurences,categorical_variable
0,NaN,0,0.0,Cultivar
1,NEO 760 CE,8,2.5,Cultivar
2,MANU IPRO,8,2.5,Cultivar
3,77HO111I2X - GUAPORÉ,8,2.5,Cultivar
4,NK 7777 IPRO,8,2.5,Cultivar
5,GNS7900 IPRO - AMPLA,8,2.5,Cultivar
6,LTT 7901 IPRO,8,2.5,Cultivar
7,BRASMAX BÔNUS IPRO,8,2.5,Cultivar
8,97Y97 IPRO,8,2.5,Cultivar
9,BRASMAX OLIMPO IPRO,8,2.5,Cultivar


### **Grouping the dataframe by a given variable**
- Categorical variables are grouped by this function only when a proper aggregation function is selected, like the 'mode'.
- If other aggregate is selected, only numeric variables are grouped.

#### Case 1: return a statistics summary dataframe

In [10]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

VARIABLES_TO_GROUP_BY = ['Season', 'Cultivar', 'Repetition']
# string (inside quotes) containing the name
# of the column in terms of which the dataframe will be grouped by.;
# or (list of strings) with the name of the columns, in case multiple columns
# are used as keys. e.g. variables_to_group_by = "column1" will group the
# dataframe in terms of 'column1'.
# WARNING: do not use this function to group a dataframe in terms of a timestamp.
# To group by a timestamp, use function group_variables_by_timestamp instead.

RETURN_SUMMARY_DATAFRAME = True
# RETURN_SUMMARY_DATAFRAME = False. Set RETURN_SUMMARY_DATAFRAME = True if you want the function
# to return a dataframe containing summary statistics (obtained with the describe method).

SUBSET_OF_COLUMNS_TO_AGGREGATE = None
# SUBSET_OF_COLUMNS_TO_AGGREGATE: list of strings (inside quotes) containing the names
# of the columns that will be aggregated. Use this argument if you want to aggregate only a subset,
# not the whole dataframe. Declare as a list even if there is a single column to group by.
# e.g. SUBSET_OF_COLUMNS_TO_AGGREGATE = ["response_feature"] will return the column
# 'response_feature' grouped. SUBSET_OF_COLUMNS_TO_AGGREGATE = ["col1", 'col2'] will return columns
# 'col1' and 'col2' grouped.
# If you want to aggregate the whole subset, keep SUBSET_OF_COLUMNS_TO_AGGREGATE = None.

AGGREGATE_FUNCTION = 'mean'
# AGGREGATE_FUNCTION = 'mean': String defining the aggregation
# method that will be applied. Possible values:
# 'median', 'mean', 'mode', 'sum', 'min', 'max', 'variance', 'count',
# 'standard_deviation', 'cum_sum', 'cum_prod', 'cum_max', 'cum_min',
# '10_percent_quantile', '20_percent_quantile',
# '25_percent_quantile', '30_percent_quantile', '40_percent_quantile',
# '50_percent_quantile', '60_percent_quantile', '70_percent_quantile',
# '75_percent_quantile', '80_percent_quantile', '90_percent_quantile',
# '95_percent_quantile', 'kurtosis', 'skew', 'interquartile_range',
# 'mean_standard_error', 'entropy'
# To use another aggregate function, you can use the .agg method, passing
# the aggregate as argument, such as in:
# .agg(scipy.stats.mode),
# where the argument is a Scipy aggregate function.
# If None or an invalid function is input, 'mean' will be used.

ADD_SUFFIX_TO_AGGREGATED_COLUMN = True
# ADD_SUFFIX_TO_AGGREGATED_COLUMN = True will add a suffix to the
# aggregated column. e.g. 'responseVar_mean'. If ADD_SUFFIX_TO_AGGREGATED_COLUMN
# = False, the aggregated column will have the original column name.
SUFFIX = None
# suffix = None. Keep it None if no suffix should be added, or if
# the name of the aggregate function should be used as suffix, after
# "_". Alternatively, set it as a string. As recommendation, put the
# "_" sign in the beginning of this string to separate the suffix from
# the original column name. e.g. if the response variable is 'Y' and
# suffix = '_agg', the new aggregated column will be named as 'Y_agg'


# Grouped dataframe, and summary statistics dataframe returned as:
# grouped_df and summary_agg_df, respectively.
# Simply modify these objects on the left of equality:
grouped_df, summary_agg_df = group_dataframe_by_variable (df = DATASET, variables_to_group_by = VARIABLES_TO_GROUP_BY, return_summary_dataframe = RETURN_SUMMARY_DATAFRAME, subset_of_columns_to_aggregate = SUBSET_OF_COLUMNS_TO_AGGREGATE, aggregate_function = AGGREGATE_FUNCTION, add_suffix_to_aggregated_col = ADD_SUFFIX_TO_AGGREGATED_COLUMN, suffix = SUFFIX)


Numeric aggregate selected. Categorical variables will be aggregated in terms of mode, the most common value.

Dataframe successfully grouped. Check its 10 first rows:



,Season,Cultivar,Repetition,PH_mean,IFP_mean,NLP_mean,NGP_mean,NGL_mean,NS_mean,MHG_mean,GY_mean
0,1,74K75RSF CE,1,63.80,15.80,56.80,139.20,2.45,5.40,167.44,2942.14
1,1,74K75RSF CE,2,65.80,13.80,73.60,180.60,2.45,5.00,158.30,2939.59
2,1,74K75RSF CE,3,77.20,17.20,56.60,149.00,2.63,4.40,159.49,2860.14
3,1,74K75RSF CE,4,68.93,15.60,62.33,156.27,2.51,4.93,158.30,2854.18
4,1,77HO111I2X - GUAPORÉ,1,52.20,14.40,64.80,148.80,2.30,7.20,180.25,3640.46
5,1,77HO111I2X - GUAPORÉ,2,55.80,15.80,72.00,188.80,2.62,6.20,176.75,3602.34
6,1,77HO111I2X - GUAPORÉ,3,54.00,19.40,78.20,182.40,2.33,7.80,173.16,3488.79
7,1,77HO111I2X - GUAPORÉ,4,54.00,16.53,71.67,173.33,2.42,7.07,176.73,3698.28
8,1,79I81RSF IPRO,1,70.00,17.80,64.00,144.60,2.26,5.00,180.08,2940.87
9,1,79I81RSF IPRO,2,68.60,15.20,77.60,179.40,2.31,5.00,175.76,2925.98




Check the summary statistics dataframe, that is also being returned:



Season              Cultivar Repetition    PH                           \
                                            count   mean std    min    25%   
0        1           74K75RSF CE          1   1.0  63.80 NaN  63.80  63.80   
1        1           74K75RSF CE          2   1.0  65.80 NaN  65.80  65.80   
2        1           74K75RSF CE          3   1.0  77.20 NaN  77.20  77.20   
3        1           74K75RSF CE          4   1.0  68.93 NaN  68.93  68.93   
4        1  77HO111I2X - GUAPORÉ          1   1.0  52.20 NaN  52.20  52.20   
..     ...                   ...        ...   ...    ...  ..    ...    ...   
315      2           SYN2282IPRO          4   1.0  64.60 NaN  64.60  64.60   
316      2          TMG 22X83I2X          1   1.0  65.20 NaN  65.20  65.20   
317      2          TMG 22X83I2X          2   1.0  66.20 NaN  66.20  66.20   
318      2          TMG 22X83I2X          3   1.0  55.40 NaN  55.40  55.40   
319      2          TMG 22X83I2X          4   1.0  62.27 NaN  62.27  62.27   

                   ...     MHG            GY                                \
       50%    75%  ...     75%     max count         mean std          min   
0    63.80  63.80  ...  167.44  167.44   1.0  2942.140000 NaN  2942.140000   
1    65.80  65.80  ...  158.30  158.30   1.0  2939.590000 NaN  2939.590000   
2    77.20  77.20  ...  159.49  159.49   1.0  2860.140000 NaN  2860.140000   
3    68.93  68.93  ...  158.30  158.30   1.0  2854.180000 NaN  2854.180000   
4    52.20  52.20  ...  180.25  180.25   1.0  3640.460000 NaN  3640.460000   
..     ...    ...  ...     ...     ...   ...          ...  ..          ...   
315  64.60  64.60  ...  204.89  204.89   1.0  3443.086207 NaN  3443.086207   
316  65.20  65.20  ...  184.11  184.11   1.0  3300.275862 NaN  3300.275862   
317  66.20  66.20  ...  184.17  184.17   1.0  3336.465517 NaN  3336.465517   
318  55.40  55.40  ...  182.95  182.95   1.0  3348.270115 NaN  3348.270115   
319  62.27  62.27  ...  183.13  183.13   1.0  3447.172414 NaN  3447.172414   

                                                         
             25%          50%          75%          max  
0    2942.140000  2942.140000  2942.140000  2942.140000  
1    2939.590000  2939.590000  2939.590000  2939.590000  
2    2860.140000  2860.140000  2860.140000  2860.140000  
3    2854.180000  2854.180000  2854.180000  2854.180000  
4    3640.460000  3640.460000  3640.460000  3640.460000  
..           ...          ...          ...          ...  
315  3443.086207  3443.086207  3443.086207  3443.086207  
316  3300.275862  3300.275862  3300.275862  3300.275862  
317  3336.465517  3336.465517  3336.465517  3336.465517  
318  3348.270115  3348.270115  3348.270115  3348.270115  
319  3447.172414  3447.172414  3447.172414  3447.172414  

[320 rows x 67 columns]

## **Exporting the dataframe as CSV file (to notebook's workspace)**

In [ ]:
## WARNING: all files exported from this function are .csv (comma separated values)

DATAFRAME_OBJ_TO_BE_EXPORTED = dataset
# Alternatively: object containing the dataset to be exported.
# DATAFRAME_OBJ_TO_BE_EXPORTED: dataframe object that is going to be exported from the
# function. Since it is an object (not a string), it should not be declared in quotes.
# example: DATAFRAME_OBJ_TO_BE_EXPORTED = dataset will export the dataset object.
# ATTENTION: The dataframe object must be a Pandas dataframe.

FILE_DIRECTORY_PATH = ""
# FILE_DIRECTORY_PATH - (string, in quotes): input the path of the directory
# (e.g. folder path) where the file is stored. e.g. FILE_DIRECTORY_PATH = ""
# or FILE_DIRECTORY_PATH = "folder"
# If you want to export the file to AWS S3, this parameter will have no effect.
# In this case, you can set FILE_DIRECTORY_PATH = None

NEW_FILE_NAME_WITHOUT_EXTENSION = "dataset"
# NEW_FILE_NAME_WITHOUT_EXTENSION - (string, in quotes): input the name of the
# file without the extension. e.g. set NEW_FILE_NAME_WITHOUT_EXTENSION = "my_file"
# to export the CSV file 'my_file.csv' to notebook's workspace.

export_pd_dataframe_as_csv (dataframe_obj_to_be_exported = DATAFRAME_OBJ_TO_BE_EXPORTED, new_file_name_without_extension = NEW_FILE_NAME_WITHOUT_EXTENSION, file_directory_path = FILE_DIRECTORY_PATH)

## **Exporting dataframes as Excel file tables**

In [ ]:
## WARNING: all files exported from this function are .xlsx

FILE_NAME_WITHOUT_EXTENSION = "datasets"
# (string, in quotes): input the name of the
# file without the extension. e.g. new_file_name_without_extension = "my_file"
# will export a file 'my_file.xlsx' to notebook's workspace.

EXPORTED_TABLES = [{'dataframe_obj_to_be_exported': None,
                    'excel_sheet_name': None},]

# exported_tables is a list of dictionaries. User may declare several dictionaries,
# as long as the keys are always the same, and if the values stored in keys are not None.

# key 'dataframe_obj_to_be_exported': dataframe object that is going to be exported from the
# function. Since it is an object (not a string), it should not be declared in quotes.
# example: dataframe_obj_to_be_exported = dataset will export the dataset object.
# ATTENTION: The dataframe object must be a Pandas dataframe.

# key 'excel_sheet_name': string containing the name of the sheet to be written on the
# exported Excel file. Example: excel_sheet_name = 'tab_1' will save the dataframe in the
# sheet 'tab_1' from the file named as file_name_without_extension.

# examples: exported_tables = [{'dataframe_obj_to_be_exported': dataset1,
# 'excel_sheet_name': 'sheet1'},]
# will export only dataset1 as 'sheet1';
# exported_tables = [{'dataframe_obj_to_be_exported': dataset1, 'excel_sheet_name': 'sheet1'},
# {'dataframe_obj_to_be_exported': dataset2, 'excel_sheet_name': 'sheet2']
# will export dataset1 as 'sheet1' and dataset2 as 'sheet2'.

# Notice that if the file does not contain the exported sheets, they will be created. If it has,
# the sheets will be replaced.

FILE_DIRECTORY_PATH = ""
# FILE_DIRECTORY_PATH - (string, in quotes): input the path of the directory
# (e.g. folder path) where the file is stored. e.g. FILE_DIRECTORY_PATH = ""
# or FILE_DIRECTORY_PATH = "folder"
# If you want to export the file to AWS S3, this parameter will have no effect.
# In this case, you can set FILE_DIRECTORY_PATH = None


export_pd_dataframe_as_excel (file_name_without_extension = FILE_NAME_WITHOUT_EXTENSION, exported_tables = EXPORTED_TABLES, file_directory_path = FILE_DIRECTORY_PATH)

****

# **Grouping by Date in Pandas - Background and Documentation**

- Suppose we have timestamps with the datetime objects stored in column 'Date' of the dataframe df.

## In the examples below, we aggregate the dataframes by date (year, month, day, min) in terms of the mean values over the set time interval.
- The time interval is the aggregation bin.
- To aggregate in terms of sum, simply substitute .mean() by .sum().
- The same is applied to the other possible aggregate functions: median, var, std, min, max, etc.
- **There are many use cases where we want the total sum over a given period of time. In those cases, we apply the .sum() aggregate** function of Pandas, instead of the .mean() used in the next examples.

### WARNING: Before grouping, make sure that the 'Date' column stores a pandas Timestamp object, with resolution of at least seconds. For that, use:
`timestamp_object = pd.Timestamp(datetime_object, unit = 's')`
- For a resolution in other scale, simply modify this parameter. For instance, unit = 'ns' for nanoseconds.
- Check the pandas.Timestamp class documentation: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timestamp.html

## Calling Grouper class
- Firstly, convert all datetime objects into pandas.Timestamps.
- To group by dates, we must call the Grouper class:
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Grouper.html

Syntax:

```
pandas.Grouper(key=None, level=None, freq=None, axis=0, sort=False)
```
- Notice that setting sort = True will sort the grouped values. We do not need to specify axis = 0, since it is the default.

## Group by Year

```
df.groupby(pd.Grouper(key='Date', freq='1Y')).mean()
```

In this case, we grouped by intervals of 1 year. We could group by different values of years, though. For instance:

```
df.groupby(pd.Grouper(key='Date', freq='2Y')).mean()
```
Groups by intervals of 2 years.

## Group by Month

```
df.groupby(pd.Grouper(key='Date', freq='1M')).mean()
```
- Again, we could modify the number of months. For instance, the aggregation by trimesters is done as:

```
df.groupby(pd.Grouper(key='Date', freq='3M')).mean()
```

## Group by Week

```
df.groupby(pd.Grouper(key='Date', freq='1W')).mean()
```
- As usual, simply modify the number before 'W' to change the number of weeks in the grouping.
- The substitution of '1W' by '2W' results in the aggregation every 2 weeks.

## Group by Day

```
df.groupby(pd.Grouper(key='Date', freq='1D')).mean()
```

- If you want to group by a different number of days, simply modify the number before 'D'.
- The group by every two days, so, is performed as `df.groupby(pd.Grouper(key='Date', freq='2D')).mean()`; whereas `df.groupby(pd.Grouper(key='Date', freq='5D')).mean()` groups by every five days.

## Group by Hour

```
grouper = df.groupby([pd.Grouper(freq='1H'), 'Location'])
```

## Group by Minute

```
df.groupby(pd.Grouper(key='Date', freq='1min')).mean()
```
- To group by every 15 mins: `df.groupby(pd.Grouper(key='Date', freq='15min')).mean()`
- To group by every 2 mins: `df.groupby(pd.Grouper(key='Date', freq='2min')).mean()`

## Group by Second

The next example upsample the time series into 30 second bins.
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.asfreq.html

```
df.asfreq(freq='30S')
```

### Adjusting the time bins based on a fixed timestamp:
- Suppose a grouping by every 17 mins.
- You can specify an origin or specify an offset (equivalent):

```
df.groupby(pd.Grouper(key='Date', freq='17min', origin='2000-01-01')).mean()
```

If the resolution of the timestamps is in days, the grouping will consider the first instant as 00:00:00. So, the following lines are completely equivalent: in the second one, we simply specified the offset in hours and minutes to not start the grouping by 00:00:00 of a given day (we specifically set the first day to start from '23h30min' after 00:00:00:

```
df.groupby(pd.Grouper(key='Date', freq='17min', origin='2000-10-01 23:30:00')).mean()
df.groupby(pd.Grouper(key='Date', freq='17min', offset='23h30min')).mean()
```
The same output can be obtained by defining a string or timestamp and passing it as argument:

```
start = '2000-10-01 23:30:00'
df.groupby(pd.Grouper(key='Date', freq='17min', origin= start)).mean()
```

Now, suppose the timestamps contain the hour information (e.g.: 01:10:20). Now, the **'offset' parameter will represent a moment for starting after the first timestamp.**
- That is because our timestamp is not necessarily 00:00:00, as before.
- When the hours are not declare, Python gives the time 00:00:00 to each timestamp.
- So, if we have `offset='2min'` the first timestamp of the grouping bins will be 2 min after the first timestamp of the dataframe df.
- Therefore, the `offset = 'XXhYYmin'` indicates to the `Grouper` class that the first bin should start with an offset of XX h and YY min in relation to the first timestamp, i.e., XX h and YY min after the first timestamp.

# **Merging (joining) the data by a timestamp with Pandas - Background and Documentation**
- We could use the .merge method, but this will not return an ordered dataframe.
- Let's use the .merge_ordered instead.
- If the data is not synchronous, we can perform the fuzzy merging using the .merge_asof method.

## Methods comparison
_From Datacamp course: Joining Data with pandas, chapter 4 - Merging Ordered and Time-Series Data_

### .merge() method:
- Column(s) to join on: on , left_on , and right_on
- Type of join: how (left, right, inner, outer) {{@}}
    - Default: 'inner'.
- Overlapping column names: suffixes
- Calling the method: df1.merge(df2)

### .merge_ordered() method:
- Column(s) to join on: on , left_on , and right_on
- Type of join: how (left, right, inner, outer)
    - Default: 'outer'.
- Overlapping column names: suffixes
- Calling the method: pd.merge_ordered(df1, df2)

Examples:

```
import pandas as pd
pd.merge_ordered(appl, mcd, on='date', suffixes=('_aapl','_mcd'))
```
#### Forward fill: fills missing with previous value

```
pd.merge_ordered(appl, mcd, on='date', suffixes=('_aapl','_mcd'), fill_method='ffill')
```
- When to use merge_ordered()?
    - Ordered data / time series.
    - Filling in missing values.

### .merge_asof() method:
- Similar to a merge_ordered() left join.
    - Similar features as merge_ordered().
- Match on the nearest key column and not exact matches.
    - Merged "on" columns must be sorted.

```
pd.merge_asof(visa, ibm, on='date_time', suffixes=('_visa','_ibm'))
```
#### merge_asof() example with direction
```
pd.merge_asof(visa, ibm, on=['date_time'], suffixes=('_visa','_ibm'), direction='forward')
```

direction: ‘backward’ (default), ‘forward’, or ‘nearest’.
-'nearest' allows both directions.
- merge_asof does not allow filling. Check:
    - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge_asof.html
    - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge_ordered.html#pandas.merge_ordered


- When to use merge_asof()
    - Data sampled from a process.
    - Developing a training set (no data leakage).
    - .merge_asof uses fuzzy matching, so the HOW parameter is not applicable.